In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm
from IPython.display import clear_output

tqdm.pandas()
pd.options.display.max_colwidth = -1
pd.options.display.max_columns = 15

In [3]:
from dsutils.eda import *
from dsutils.stats import *
from dsutils.ml import *

In [4]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
STOP_WORDS = stopwords.words("english")
ps = PorterStemmer()

In [5]:
def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    x = re.sub(r'\W', ' ', str(x))
    
    x = re.sub(r"\s+", ' ', str(x))
    
    x = ' '.join(ps.stem(word) for word in x.split())
    
    soup = BeautifulSoup(str(x))
    x = soup.get_text()

    x = ' '.join([word for word in x.split() if word not in STOP_WORDS])
    
    return x

In [6]:
data = pd.read_csv('basic_features.csv')

In [7]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,abs_token_diff,avg_num_token,rel_token_diff,token_intersection,token_union,jaccard_similarity_token,lcs_token,lcs_token_ratio
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0,2,13.0,0.153846,10,13,0.769231,11,0.846154
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0,5,10.5,0.476190,4,16,0.250000,4,0.380952
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0,4,12.0,0.333333,3,21,0.142857,3,0.250000
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0,2,10.0,0.200000,0,19,0.000000,0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0,6,10.0,0.600000,2,18,0.111111,2,0.200000


In [8]:
data['q1_clean'] = data['question1'].progress_apply(preprocess)
data['q2_clean'] = data['question2'].progress_apply(preprocess)

In [9]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,abs_token_diff,...,token_intersection,token_union,jaccard_similarity_token,lcs_token,lcs_token_ratio,q1_clean,q2_clean
0,0,1,2,What is the step by step guide to invest in share market in india?,What is the step by step guide to invest in share market?,0,2,...,10,13,0.769231,11,0.846154,step step guid invest share market india,step step guid invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Diamond?,What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?,0,5,...,4,16,0.250000,4,0.380952,stori kohinoor koh noor diamond,would happen indian govern stole kohinoor koh noor diamond back
2,2,5,6,How can I increase the speed of my internet connection while using a VPN?,How can Internet speed be increased by hacking through DNS?,0,4,...,3,21,0.142857,3,0.250000,increas speed internet connect use vpn,internet speed increas hack dn
3,3,7,8,Why am I mentally very lonely? How can I solve it?,"Find the remainder when [math]23^{24}[/math] is divided by 24,23?",0,2,...,0,19,0.000000,0,0.000000,whi mental veri lone solv,find remaind math 23 24 math divid 24 23
4,4,9,10,"Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?",Which fish would survive in salt water?,0,6,...,2,18,0.111111,2,0.200000,one dissolv water quikli sugar salt methan carbon di oxid,fish would surviv salt water


In [10]:
data.to_csv('train_cleaned.csv', index=False)